# Zoom 출석 체크 자동화 도구 (느슨한 매칭 버전)

이 노트북은 Zoom 로그와 출석 명단 파일을 기반으로 다음을 수행합니다:

- Zoom 이름에 **학번 또는 이름이 포함**되어 있으면 출석으로 인식
- 출석 명단 파일의 **지정한 날짜 열**에 '출석' 표시
- Zoom 이름과 매칭되지 않은 인원은 별도 시트로 정리

---

## ✅ 사용법
1. `출석명단 최종.csv` 와 `zoom 로그.csv` 파일을 같은 폴더에 둡니다
2. 이 노트북을 열고, 실행 전 `출석 열 이름`을 원하는 날짜로 수정합니다 (예: `"4월21일"`)
3. 모든 셀을 위에서 아래로 실행하면 `출석결과_느슨한매칭_자동.xlsx` 파일이 생성됩니다


In [ ]:
import pandas as pd

In [ ]:
# 여기만 수정하세요 (출석 열 이름 예: "4월21일")
출석열이름 = "4월21일"


In [ ]:
attendance_base_df = pd.read_csv("출석명단 최종.csv")
zoom_log_df = pd.read_csv("zoom 로그.csv")

attendance_clean = attendance_base_df.iloc[2:].reset_index(drop=True)
attendance_clean.columns = ["", "과정명", "NO", "순번", "성명", "학번", 출석열이름]
attendance_clean = attendance_clean[["성명", "학번", 출석열이름]].copy()
attendance_clean["학번"] = attendance_clean["학번"].astype(str).str.strip()
attendance_clean["성명"] = attendance_clean["성명"].astype(str).str.strip()


In [ ]:
zoom_log_df["zoom_이름전체"] = zoom_log_df["이름(원래 이름)"].astype(str)

def is_loose_match(row, zoom_names):
    student_id = str(row["학번"]).strip()
    student_name = str(row["성명"]).strip()
    for zoom_name in zoom_names:
        if student_id in zoom_name or student_name in zoom_name:
            return True
    return False

zoom_name_list = zoom_log_df["zoom_이름전체"].tolist()
attendance_clean["매칭됨"] = attendance_clean.apply(lambda row: is_loose_match(row, zoom_name_list), axis=1)
matched_students = attendance_clean[attendance_clean["매칭됨"] == True]

def match_any_student(row):
    for _, s_row in matched_students.iterrows():
        if str(s_row["학번"]) in row["zoom_이름전체"] or str(s_row["성명"]) in row["zoom_이름전체"]:
            return f"{s_row['학번']}|{s_row['성명']}"
    return None

zoom_log_df["식별자"] = zoom_log_df.apply(match_any_student, axis=1)


In [ ]:
matched_logs_v2 = zoom_log_df.dropna(subset=["식별자"])
time_sum_df_v2 = matched_logs_v2.groupby("식별자")["기간(분)"].sum().reset_index()
time_sum_df_v2["출석 여부"] = time_sum_df_v2["기간(분)"] >= 90
time_sum_df_v2["출석 여부"] = time_sum_df_v2["출석 여부"].map({True: "출석", False: ""})

attendance_clean["식별자"] = attendance_clean.apply(lambda row: f"{row['학번']}|{row['성명']}", axis=1)
final_attendance_v2 = attendance_clean.merge(time_sum_df_v2, on="식별자", how="left")
final_attendance_v2[출석열이름] = final_attendance_v2["출석 여부"].fillna("")


In [ ]:
matched_ids_v2 = set(time_sum_df_v2["식별자"])
unmatched_logs_v2 = zoom_log_df[~zoom_log_df["식별자"].isin(matched_ids_v2)]
unmatched_summary_v2 = unmatched_logs_v2.groupby("zoom_이름전체")["기간(분)"].sum().reset_index()
unmatched_summary_v2.rename(columns={"zoom_이름전체": "Zoom 표시 이름", "기간(분)": "총 체류 시간(분)"}, inplace=True)
unmatched_summary_v2["비고"] = "이름/학번 모두 미일치"

with pd.ExcelWriter("출석결과_느슨한매칭_자동.xlsx", engine="openpyxl") as writer:
    final_attendance_v2[["성명", "학번", 출석열이름]].to_excel(writer, index=False, sheet_name="출석 체크")
    unmatched_summary_v2.to_excel(writer, index=False, sheet_name="확인 필요")

print("✅ 엑셀 저장 완료: 출석결과_느슨한매칭_자동.xlsx")